Antonio Fernandes Valadares - 11711ECP015 \
Computação Evolutiva \
Universidade Federal de Uberlândia - Engenharia de computação \
Professor: Keiji Yamanaka \
Trabalho 4 : Manutenção de sistema elétrico de potência

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

pl = reserva líquida \
pt = potência total instalada \
pp = potência perdida por parada \
pd = potência máxima de demanda \

pl = pt - pp - pd

Critério ótimo: pl deve ser a máxima em cada intervalo. \

Restrições: pl deve ser maior ou igual a zero em qualquer intervalo. \
A manuteção de uma máquina começa no início de um intervalor e termina no final do mesmo.

In [3]:
carac = pd.DataFrame({'num da maquina' : [1,2,3,4,5,6,7],
                     'capacidade' : [20,15,35,40,15,15,10],
                     'intervalos de manunteção/ano' : [2,2,1,1,1,1,1]})

In [4]:
# Características das máquinas

carac

,num da maquina,capacidade,intervalos de manunteção/ano
0,1,20,2
1,2,15,2
2,3,35,1
3,4,40,1
4,5,15,1
5,6,15,1
6,7,10,1


In [5]:
# Demanda máxima por intervalo

demanda = pd.DataFrame({'intervalo' : [1,2,3,4],
                       'pd' : [80,90,65,70]})

demanda

,intervalo,pd
0,1,80
1,2,90
2,3,65
3,4,70


In [6]:
# Potência total

pt = carac['capacidade'].sum()
pt

150

In [7]:
# função a ser maximizada???

def calc_pl(pp, pd):
  pl = pt - pp - pd

  return pl

In [8]:
# Como gera o código genético de um individuo?

#### individuo = [1,2],[3,4],[1],[2],[3],[4],[3],[1]

# 00 - intervalo 1, 01- intervalo 2, 10 - intervalo 3, 11 - intervalo 4
# 000 - intervalo [1,2]
# 001 - intervalo [1,3]
# 010 - intervalo [1,4]
# 011 - intervalo [2,3]
# 100 - intervalo [2,4]
# 101 - intervalo [3,4]

dict_intervalos = {'00' : [1],
                   '01' : [2],
                   '10' : [3],
                   '11' : [4],
                   '000' : [1,2],
                   '001' : [1,3],
                   '010' : [1,4],
                   '011' : [2,3],
                   '100' : [2,4],
                   '101' : [3,4]}


# EX: 001 / 011 / 00 / 01 / 10 / 11 / 10 / 00 

In [9]:
def initial_population(size = 20):
  d = {'1' : np.random.randint(6, size = size),
       '2' : np.random.randint(6, size = size),
       '3' : np.random.randint(4, size = size),
       '4' : np.random.randint(4, size = size),
       '5' : np.random.randint(4, size = size),
       '6' : np.random.randint(4, size = size),
       '7' : np.random.randint(4, size = size)}


  population = pd.DataFrame(data=d, columns=['1', '2', '3', '4', '5', '6', '7'])
  population['1'] = population['1'].apply(lambda x: '{0:03b}'.format(x))
  population['2'] = population['2'].apply(lambda x: '{0:03b}'.format(x))
  for i in range(3,8):
    population[f'{i}'] = population[f'{i}'].apply(lambda x: '{0:02b}'.format(x))

  population['cromossomo'] = population['1'] + population['2'] + population['3'] + population['4'] + population['5'] + population['6'] + population['7']

  return population, population['cromossomo'].values


population, cromossomos = initial_population(size=10)
population

,1,2,3,4,5,6,7,cromossomo
0,000,101,10,10,00,11,01,0001011010001101
1,010,001,01,00,01,10,01,0100010100011001
2,011,100,10,10,11,00,01,0111001010110001
3,001,011,00,00,01,00,00,0010110000010000
4,101,100,10,10,11,00,10,1011001010110010
5,010,001,11,01,11,10,10,0100011101111010
6,010,011,00,10,00,01,11,0100110010000111
7,011,011,10,11,00,10,00,0110111011001000
8,010,011,11,00,01,00,00,0100111100010000
9,100,000,11,10,01,00,00,1000001110010000


In [21]:
# Qual função usar para calcular a aptidão de cada individuo?
# Como calcular o valor de pl para cada intervalo?

def std(x):
  return np.std([x['pl1'],x['pl2'],x['pl3'],x['pl4']])

def fitness(population):
  population['pl1'] = pt - 80
  population['pl2'] = pt - 90
  population['pl3'] = pt - 65
  population['pl4'] = pt - 70

  population.loc[(population['1'] == '000') | (population['1'] == '001') | (population['1'] == '010'), 'pl1'] = population['pl1'] - carac.iloc[0,1]
  population.loc[(population['2'] == '000') | (population['2'] == '001') | (population['2'] == '010'), 'pl1'] = population['pl1'] - carac.iloc[1,1]

  population.loc[(population['1'] == '000') | (population['1'] == '011') | (population['1'] == '100'), 'pl2'] = population['pl2'] - carac.iloc[0,1]
  population.loc[(population['2'] == '000') | (population['2'] == '011') | (population['2'] == '100'), 'pl2'] = population['pl2'] - carac.iloc[1,1]

  population.loc[(population['1'] == '001') | (population['1'] == '011') | (population['1'] == '101'), 'pl3'] = population['pl3'] - carac.iloc[0,1]
  population.loc[(population['2'] == '001') | (population['2'] == '011') | (population['2'] == '101'), 'pl3'] = population['pl3'] - carac.iloc[1,1]

  population.loc[(population['1'] == '010') | (population['1'] == '100') | (population['1'] == '101'), 'pl4'] = population['pl4'] - carac.iloc[0,1]
  population.loc[(population['2'] == '010') | (population['2'] == '100') | (population['2'] == '101'), 'pl4'] = population['pl4'] - carac.iloc[1,1]

  for i in range(3,8):
    population.loc[population[f'{i}'] == '00', 'pl1'] = population['pl1'] - carac.iloc[i-1, 1]
    population.loc[population[f'{i}'] == '01', 'pl2'] = population['pl2'] - carac.iloc[i-1, 1]
    population.loc[population[f'{i}'] == '10', 'pl3'] = population['pl3'] - carac.iloc[i-1, 1]
    population.loc[population[f'{i}'] == '11', 'pl4'] = population['pl4'] - carac.iloc[i-1, 1]

  population['fitness'] = population.apply(std, axis=1)
  population.sort_values(by='fitness', inplace=True)

  best = population.iloc[0]
  print('Melhor resultado: ', best['cromossomo'], '\n Itervalos máquina 1:', dict_intervalos.get(best['1']))
  print(' Itervalos máquina 2:', dict_intervalos.get(best['2']))
  print(' Itervalos máquina 3:', dict_intervalos.get(best['3']))
  print(' Itervalos máquina 4:', dict_intervalos.get(best['4']))
  print(' Itervalos máquina 5:', dict_intervalos.get(best['5']))
  print(' Itervalos máquina 6:', dict_intervalos.get(best['6']))
  print(' Itervalos máquina 7:', dict_intervalos.get(best['7']))
  print(' Margem de segurança dos intervalos:',  best['pl1'], best['pl2'], best['pl3'], best['pl4'], ' Desvio padrão: ', best['fitness'])

  #return np.array([population['cromossomo'], population['fitness']])
  return population

population, _ = initial_population(size = 10)
population = fitness(population)
population

Melhor resultado:  1010111100100110 
 Itervalos máquina 1: [3, 4]
 Itervalos máquina 2: [2, 3]
 Itervalos máquina 3: [4]
 Itervalos máquina 4: [1]
 Itervalos máquina 5: [3]
 Itervalos máquina 6: [2]
 Itervalos máquina 7: [3]
 Margem de segurança dos intervalos: 30 30 25 25  Desvio padrão:  2.5


,1,2,3,4,5,6,7,cromossomo,pl1,pl2,pl3,pl4,fitness
5,101,011,11,00,10,01,10,1010111100100110,30,30,25,25,2.500000
2,101,001,00,01,11,10,10,1010010001111010,20,20,25,45,10.307764
6,100,010,10,00,10,10,11,1000101000101011,15,40,20,35,10.307764
9,001,100,01,11,00,10,11,0011000111001011,35,10,50,15,16.007811
1,011,101,10,11,11,01,11,0111011011110111,70,25,15,0,26.100766
7,011,000,11,00,00,00,10,0110001100000010,-15,25,55,45,26.809513
0,001,011,01,00,01,11,10,0010110100011110,10,-5,40,65,27.041635
3,100,010,01,11,01,10,01,1000100111011001,55,-20,70,5,36.486299
4,100,011,01,11,00,01,01,1000110111000101,55,-35,70,20,40.388736
8,011,011,01,01,01,00,00,0110110101010000,45,-65,50,80,55.056789


In [11]:
#population, _ = initial_population(size = 10)
#cromossomos = fitness(population)

def torneio(cromossomos, k, size):

  p = np.random.choice(np.arange(0,size), size=k)

  c = np.array([cromossomos[:,p[i]] for i in range(k)])

  c = c[c[:, 1].argsort()]

  return c[0,0]


def crossover(cromossomos, size, crossover_rate=0.6, k=3, elit=1):
  new_population = []

  for i in range(0,elit):
    new_population.append(cromossomos[0,i])

  while(len(new_population) < size):
    cp1 = torneio(cromossomos, k, size)
    cp2 = torneio(cromossomos, k, size)


    if(np.random.random() < crossover_rate):
      i = random.randint(0,len(cp1))

      cf1 = cp1[0:i] + cp2[i:16]
      cf2 = cp2[0:i] + cp1[i:16]

      new_population.append(cf1)
      new_population.append(cf2)


    else:
      new_population.append(cp1)
      new_population.append(cp2)

  del new_population[-1]
  new_population = np.array(new_population)

  return new_population


#crossover(cromossomos, size=10, elit=1)


In [12]:
def mut(x):
  x = list(x)
  i = random.randint(0,len(x)-1)
  if(x[i] == '0'):
    x[i] = '1'
  else:
    x[i] = '0'
    
  if(x[0] == '1' and x[1] == '1'):
    x[0] = '0'

  if(x[3] == '1' and x[4] == '1'):
    x[3] = '0'

  return ''.join(x)

def mutation(cromossomos, mutation_rate=0.1):
  
  new_cromossomos = []
  new_cromossomos.append(cromossomos[0])
  for element in cromossomos[1:]:
    if(np.random.random()<mutation_rate):
      new_cromossomos.append(mut(element))
    else:
      new_cromossomos.append(element)

  return new_cromossomos

#population, _ = initial_population(size = 10)
#cromossomos = fitness(population)
#cromossomos_c = crossover(cromossomos, size=10, elit=1)
#cromossomos_m = mutation(cromossomos_c, mutation_rate=0.6)

In [13]:
def genetic_algorithm(size=20, crossover_rate=0.6, k=3, geracoes=100, mutation_rate=0.1):
  population, _ = initial_population(size = 10)
  new_population = pd.DataFrame().reindex_like(population).dropna()

  i = 0
  while(i+1 < geracoes):
    print('\nGeração',i,': \n')

    cromossomos = fitness(population)
    cromossomos_c = crossover(cromossomos, size=10, elit=1)
    cromossomos_m = mutation(cromossomos_c, mutation_rate=0.6)


    new_population['cromossomo'] = cromossomos_m
    new_population['1'] = new_population['cromossomo'].apply(lambda x: x[0:3])
    new_population['2'] = new_population['cromossomo'].apply(lambda x: x[3:6])
    new_population['3'] = new_population['cromossomo'].apply(lambda x: x[6:8])
    new_population['4'] = new_population['cromossomo'].apply(lambda x: x[8:10])
    new_population['5'] = new_population['cromossomo'].apply(lambda x: x[10:12])
    new_population['6'] = new_population['cromossomo'].apply(lambda x: x[12:14])
    new_population['7'] = new_population['cromossomo'].apply(lambda x: x[14:16])
    population = new_population
    i = i + 1

#genetic_algorithm(size=10, geracoes=100)

In [ ]:
genetic_algorithm(size=10, geracoes=100)